## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mar Del Plata,AR,-38.0023,-57.5575,77.02,67,0,21.00,clear sky
1,1,Saskylakh,RU,71.9167,114.0833,-19.82,100,100,3.78,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,44.26,87,100,4.61,light rain
3,3,Provideniya,RU,64.3833,-173.3000,4.86,86,24,4.00,few clouds
4,4,Marshall,US,34.3834,-86.3333,31.48,51,0,1.14,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
customer_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
customer_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mar Del Plata,AR,-38.0023,-57.5575,77.02,67,0,21.00,clear sky
6,6,Busselton,AU,-33.6500,115.3333,77.04,60,7,13.00,clear sky
8,8,Mataura,NZ,-46.1927,168.8643,56.73,99,100,2.64,light rain
11,11,Port Lincoln,AU,-34.7333,135.8667,70.88,73,9,13.89,clear sky
12,12,Te Anau,NZ,-45.4167,167.7167,57.42,93,100,0.72,overcast clouds
17,17,Kaeo,NZ,-35.1000,173.7833,74.32,67,100,12.64,overcast clouds
18,18,Puerto Ayora,EC,-0.7393,-90.3518,73.96,82,34,6.33,scattered clouds
19,19,Port Alfred,ZA,-33.5906,26.8910,68.45,87,11,8.03,few clouds
21,21,New Norfolk,AU,-42.7826,147.0587,76.66,30,0,2.93,clear sky
24,24,Cape Town,ZA,-33.9258,18.4232,70.21,69,0,3.00,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
customer_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = customer_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mar Del Plata,AR,77.02,clear sky,-38.0023,-57.5575,
6,Busselton,AU,77.04,clear sky,-33.6500,115.3333,
8,Mataura,NZ,56.73,light rain,-46.1927,168.8643,
11,Port Lincoln,AU,70.88,clear sky,-34.7333,135.8667,
12,Te Anau,NZ,57.42,overcast clouds,-45.4167,167.7167,
17,Kaeo,NZ,74.32,overcast clouds,-35.1000,173.7833,
18,Puerto Ayora,EC,73.96,scattered clouds,-0.7393,-90.3518,
19,Port Alfred,ZA,68.45,few clouds,-33.5906,26.8910,
21,New Norfolk,AU,76.66,clear sky,-42.7826,147.0587,
24,Cape Town,ZA,70.21,clear sky,-33.9258,18.4232,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping") 

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [17]:
# 7. Drop the rows where there is no Hotel Name.
#find the rows where no Hotel name pulled - store index numbers for removal. 
no_hotels = hotel_df[hotel_df['Hotel Name'] == ''].index
#drop no hotel rows from DF by index number
clean_hotel_df = hotel_df.drop(no_hotels)
#check dataframe for cleaned data
# clean_hotel_df.tail(20)

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))